In [1]:
import numpy as np
import math

## A.1 日時

- 日付`NDay`は1/1からの通しの日数
- 時刻`NHour`は整数, 0時～23時が基本。12/31のみ24時あり
- 時刻`TT`は、時間分割`MM`における時刻
  - `MM`：1時間の内の$(1/$ `NDT`$)$間隔の順番, 正時が`MM`$=0$, `MM`$=0～$ `NDT`$-1$


- 時刻`Hour01`は、前時刻の`MM` $=$ `NDT`$/2～$同時刻の`MM` $=$ `NDT`$/2-1$の時間分割を、その正時に属するものとして扱うための時刻 → 時間毎の効果係数を算定するために使用するだけ

In [2]:
def calc_NDayNHour(Hour00):
    if Hour00==8760:
        [NDay, NHour] = [365, 24]
    else:
        NDay = Hour00//24 + 1
        NHour = Hour00 - (NDay - 1) * 24 

    return [NDay,NHour]

In [3]:
def calc_TT(NHour, NDT, MM):
    return NHour + MM / float(NDT)

In [4]:
def calc_Hour01(TT):
    return int(TT + 0.5)

## 均時差と赤緯

### 1) 赤坂の式

参考：拡張アメダス気象データ1981-2000解説書 8.1　太陽位置の計算

#### Outline

$$
\displaystyle
n = Y - 1968
$$

$$
\displaystyle
d_0 = 3.71 + 0.2596 \cdot n - INT \biggl[ \frac{ n + 3 }{ 4 } \biggr]
$$

$$
\displaystyle
M = \frac{ 360 \cdot (D - d_0) }{ D_{ay} }
$$

$$
\displaystyle
\epsilon = 12.3901 + 0.0172 \cdot \biggl( n + \frac{ M }{ 360 } \biggr)
$$

$$
\displaystyle
v = M + 1.914 \cdot \sin M + 0.02 \cdot \sin(2M)
$$

$$
\displaystyle
E_t = \left ( (M - v) - \tan^{-1} \left( \frac{ 0.043 \cdot \sin 2( v + \epsilon ) }{ 1 - 0.043 \cdot \cos 2(v + \epsilon) } \right) \right) \div 15
$$

$$
\displaystyle
\sin \delta = \cos ( v + \epsilon ) \cdot \sin \delta_0
$$

$INT$ 小数点以下切り捨てを表す

Input:  
$Y$ is year / 西暦年  
$D$ is total day / 年通算日(1/1を $D=1$ とする), d    

$n$ is the difference of year from 1968 / 1968年との年差  
$d_0$ is 平均軌道上の近日点通過日（暦表時による1968年1月1日正午基準の日差）, d    
$D_{ay}$ is anomalistic year / 近点年（近日点基準の公転周期日数） = 365.2596, d    
$M$ is mean anomaly / 平均近点離角, deg  
$\epsilon$ is angle between perihelion and winter solstitial point / 近日点と冬至点の角度, deg   
$v$ is true anomaly / 真近点離角, deg  
$E_t$ is equation of time / 均時差, h  
$\delta_0$ is daily declination of winter solstice of northern hemisphere / 北半球の冬至の日赤緯 = -23.4393, deg    
$\delta$ is declination / 赤緯, rad  

#### Function

In [5]:
def get_eqation_of_time_and_declination_AKASAKA(year, nday):

    # anomalistic year / 近点年
    D_AY = 365.2596

    # daily declination of winter solstice of northern hemisphere / 北半球の冬至の日赤緯
    DLT0 = math.radians(-23.4393)
    
    # the difference of year from 1968 / 1968年との年差  
    n = year - 1968
    
    # 平均軌道上の近日点通過日（暦表時による1968年1月1日正午基準の日差）, d        
    d0 = 3.71 + 0.2596 * n - int((n + 3) / 4)    
    
    # mean anomaly / 平均近点離角, deg  
    M = 360. * (nday - d0) / D_AY
    
    # angle between perihelion and winter solstitial point / 近日点と冬至点の角度, deg   
    eps = 12.3901 + 0.0172 * (n + M / 360)
    
    # true anomaly / 真近点離角, deg  
    v = M + 1.914 * math.sin(math.radians(M)) \
        + 0.02 * math.sin(math.radians(2. * M))

    veps = math.radians(v + eps)
    
    # equation of time / 均時差, h  
    et = (M - v) \
         - math.degrees(math.atan(
                        0.043 * math.sin(2. * veps)
                        / (1.0 - 0.043 * math.cos(2.0 * veps))
                        ))
    et_hour = et / 15.0
    
    # declination / 赤緯, rad  
    sindlt = math.cos(veps) * math.sin(DLT0)
    cosdlt = (abs(1. - sindlt**2)) ** 0.5
    dlt = math.atan2(sindlt, cosdlt)
    
    return et_hour, dlt

## A.2 赤緯の計算 (仕様書6.2 式(4))

$ \begin{align}
\delta_d = (180 / \pi) & \{0.006322 - 0.405748 \cos (2 \pi N / 366 + 0.153231)\\
& - 0.005880 \cos (4 \pi N / 366 + 0.207099)\\
& - 0.003233 \cos (6 \pi N / 366 + 0.620129) \}
\end{align} $

$\delta_d$: 赤緯, deg  
$N$: 1月1日を$N=1$とした年頭からの通しの日数, d  

右辺の余弦のかっこ内の角度は$radian$単位となっているので注意

In [6]:
def calc_deltad(NDay):
    return (180 / math.pi) * (0.006322 - 0.405748 * math.cos(2 * math.pi * float(NDay) / 366 + 0.153231)
                                       - 0.005880 * math.cos(4 * math.pi * float(NDay) / 366 + 0.207099)
                                       - 0.003233 * math.cos(6 * math.pi * float(NDay) / 366 + 0.620129))

## A.3 均時差の計算 (仕様書6.2 式(6))

$ \begin{align}
e_d = -0.000279 &+ 0.122772 \cos (2 \pi N / 366 + 1.498311)\\
& - 0.165458 \cos (4 \pi N / 366 - 1.261546)\\
& - 0.005354 \cos (6 \pi N / 366 - 1.1571) \}
\end{align} $

$e_d$: 均時差, h  
$N$: 1月1日を$N=1$とした年頭からの通しの日数, d

右辺の余弦のかっこ内の角度は$radian$単位となっているので注意

In [7]:
def calc_eed(NDay):
    return ( -0.000279 + 0.122772 * math.cos(2 * math.pi * NDay / 366 + 1.498311)
                       - 0.165458 * math.cos(4 * math.pi * NDay / 366 - 1.261546)
                       - 0.005354 * math.cos(6 * math.pi * NDay / 366 - 1.1571)   )

### 時角

$$
\displaystyle
T = (t + e_d - 12) \times 15 + (L - L_0)
$$

$T$ is hour angle / 時角, rad  
$t$ is time / 時刻, h  
$e_d$ is equation of time / 均時差, h  
$L$ is longitude 経度, rad  
$L_0$ is the longitude of the location where the local time defined / 標準時の地点の経度（=135.0 degree （日本の場合））, rad

In [8]:
def calc_Tdt(longitude, eed, TT, longitude_std):
    return (TT + eed - 12) * math.radians(15.0) + longitude - longitude_std

### 太陽高度

#### Outline

$$
\displaystyle
\sin h_S = \max(0, \sin \phi \sin \delta + \cos \phi \cos \delta \cos T)
$$

$$
\displaystyle
\cos h_S = (1 - \sin ^2 h_S)^{0.5}
$$

$h_S$ is solar altitude / 太陽高度, rad  
$\phi$ is latitude, 緯度, rad  
$\delta$ is declination / 赤緯, rad   
$T$ is hour angle / 時角, rad

#### Function

In [9]:
def calc_hs(latitude, delta, t):

    sin_hs = max(0.0,
               math.sin(latitude) * math.sin(delta) \
               + math.cos(latitude) * math.cos(delta) * math.cos(t)
              )
    
    cos_hs = (1 - sin_hs**2) **0.5
    
    if sin_hs == 1.0:
        hs = math.radians(90.0)
    else:
        hs = math.atan2(sin_hs, cos_hs)
    
    return hs, sin_hs, cos_hs

### 太陽方位角

三浦コメント：  
分母が0の場合を回避しなくてもよいのか？  
高度が0のときでも太陽方位角は定義できるので,h>0 で始めるif文は不要では。（西澤さんの式にはなかった。児島さんの式にはあった。）  hs の計算の時点で0より大の制約をいれている以上、ここでも何らかの処理は必要かと思う。あるいは、hs の計算の時点では0より大の処理は含めず、全体計算でhs>0 の場合わけを行うか？

#### Outline

$$
\displaystyle
\sin A_{ZS} = \frac{ \cos \delta \sin T }{ \cos h_S }
$$

$$
\displaystyle
\cos A_{ZS} = \frac { \sin h_S \sin \phi - \sin \delta }{ \cos h_S \cos \phi }
$$

$A_{ZS}$ is solar azimuth / 太陽方位角, rad  
$h_S$ is solar altitude / 太陽高度, rad  
$\delta$ is declination / 赤緯, rad  
$T$ is hour angle / 時角, rad  
$\phi$ is latitude / 緯度, rad

#### Function

In [10]:
def calc_Azs(latitude, delta, t, hs):
    
    sinAzs = math.cos(delta) * math.sin(t) / math.cos(hs)
    cosAzs = (math.sin(hs) * math.sin(latitude) - math.sin(delta)) \
             / (math.cos(hs) * math.cos(latitude))
    
    if hs > 0.0:
        if sinAzs == 1.0:
            return math.radians(90.0)
        elif sinAzs == -1.0:
            return math.radians(270.0)
        else:
            return math.atan2(sinAzs, cosAzs)
    else:
        return 0.0

## A.8 窓面の方位 (仕様書5.2 図4)

- 窓面の方位は、以下の通り
  - 北北東：$-157.5°$, 北東：$-135°$, …, 東：$-90°$, …, 南：$0°$, …, 西：$+90°$, …,北：$+180°$
  - 角度指定も可：$-180°< A_{ZW,j} \leq +180°$
  - デフォルトは8方位指定

In [11]:
def calc_Azwj(Azimuth):
    
    Azimuth00 = ["北北東", "北東", "東北東", "東", "東南東", "南東", "南南東", "南"
                 , "南南西", "南西", "西南西", "西", "西北西", "北西", "北北西", "北" ]
    if Azimuth in Azimuth00:
        return (Azimuth00.index(Azimuth) - 7) * 22.5
    elif -180 < float(Azimuth) <= 180:
        return float(Azimuth) 
    else:
        raise ValueError('窓面方位の入力が不適切です')

## A.9 窓面の法線ベクトルと太陽位置とのなす水平面上の角度の計算 (仕様書6.2 式(1))

- 窓面の法線ベクトルと太陽位置とのなす水平面上の角度$A_{ZW,j,d,t}[deg]$, 太陽方位角$A_{ZS,d,t}[deg]$, 外壁$j$の方位角$A_{ZW,j}[deg]$

$$
A_{ZW,j,d,t} = \left\{
\begin{array}{ll}
A_{ZS,d,t} - A_{ZW,j} \hspace{48pt} (-180 < A_{ZS,d,t} - A_{ZW,j} \leq 180)
\\
A_{ZS,d,t} - A_{ZW,j} + 360 \hspace{24pt} (A_{ZS,d,t} - A_{ZW,j} \leq -180)
\\
A_{ZS,d,t} - A_{ZW,j} - 360 \hspace{24pt} (A_{ZS,d,t} - A_{ZW,j} \geq 180)
\end{array}
\right.  \qquad (1) 
$$

In [12]:
def calc_Azwjdt(Azwj, Azsdt):
    
    Azwjdt = Azsdt - Azwj
    if Azwjdt < -180:
        return Azwjdt + 360.0
    elif Azwjdt > 180:
        return Azwjdt - 360.0
    else:
        return Azwjdt

## A.10 窓まわり寸法のデータの持たせ方デフォルト (仕様書5.1 図2)

In [13]:
def set_WSSize(WSSize1):
    
    WSSizeDict = {}
    
    for i in range(0, len(WSSize1), 2):
        WSSizeDict[WSSize1[i]] = WSSize1[i+1]

    # X1
    if "X1" not in WSSizeDict:
        WSSizeDict["X1"] = 0
    elif WSSizeDict["X1"] < 0:
        raise ValueError("寸法X1の設定が不適切です")
    elif WSSizeDict["X1"] == "":
        WSSizeDict["X1"] = 0        
        
    # X2
    if "X2" not in WSSizeDict:
        raise ValueError("寸法X2が設定されていません")
    elif WSSizeDict["X2"] <= 0 or WSSizeDict["X2"] == "":
        raise ValueError("寸法X2の設定が不適切です")            
        
    # X3
    if "X3" not in WSSizeDict:
        WSSizeDict["X3"] = 0
    elif WSSizeDict["X3"] < 0:
        raise ValueError("寸法X3の設定が不適切です")
    elif WSSizeDict["X3"] == "":
        WSSizeDict["X3"] = 0 
        
    # Y1
    if "Y1" not in WSSizeDict:
        WSSizeDict["Y1"] = 0
    elif WSSizeDict["Y1"] < 0:
        raise ValueError("寸法Y1の設定が不適切です")
    elif WSSizeDict["Y1"] == "":
        WSSizeDict["Y1"] = 0 
        
    # Y2
    if "Y2" not in WSSizeDict:
        raise ValueError("寸法Y2が設定されていません")
    elif WSSizeDict["Y2"] <= 0 or WSSizeDict["Y2"] == "":
        raise ValueError("寸法Y2の設定が不適切です")            

    # Y3
    if "Y3" not in WSSizeDict:
        WSSizeDict["Y3"] = 0
    elif WSSizeDict["Y3"] < 0:
        raise ValueError("寸法Y3の設定が不適切です")            
    elif WSSizeDict["Y3"] == "":
        WSSizeDict["Y3"] = 0 
        
    # Zxp
    if "Zxp" not in WSSizeDict:
        WSSizeDict["Zxp"] = 0
    elif WSSizeDict["Zxp"] < 0:
        raise ValueError("寸法Zxpの設定が不適切です")
    elif WSSizeDict["Zxp"] == "":
        WSSizeDict["Zxp"] = 0 
        
    # Zxm
    if "Zxm" not in WSSizeDict:
        WSSizeDict["Zxm"] = 0
    elif WSSizeDict["Zxm"] < 0:
        raise ValueError("寸法Zxmの設定が不適切です")           
    elif WSSizeDict["Zxm"] == "":
        WSSizeDict["Zxm"] = 0 
        
    # Zyp
    if "Zyp" not in WSSizeDict:
        WSSizeDict["Zyp"] = 0
    elif WSSizeDict["Zyp"] < 0:
        raise ValueError("寸法Zypの設定が不適切です")     
    elif WSSizeDict["Zyp"] == "":
        WSSizeDict["Zyp"] = 0 
        
    # Zym
    if "Zym" not in WSSizeDict:
        WSSizeDict["Zym"] = 0
    elif WSSizeDict["Zym"] < 0:
        raise ValueError("寸法Zymの設定が不適切です")
    elif WSSizeDict["Zym"] == "":
        WSSizeDict["Zym"] = 0 
        
    """ 以下、オプション扱い → 非入力は窓端から日よけの付け根までの距離とする """     
    # X1yp
    if "X1yp" not in WSSizeDict:
        WSSizeDict["X1yp"] = WSSizeDict["X1"] 
    elif WSSizeDict["X1yp"] < 0:
        raise ValueError("寸法X1ypの設定が不適切です")        
    elif WSSizeDict["X1yp"] > WSSizeDict["X1"] or WSSizeDict["X1yp"] == "":
        WSSizeDict["X1yp"] = WSSizeDict["X1"]         
        
    # X1ym
    if "X1ym" not in WSSizeDict:
        WSSizeDict["X1ym"] = WSSizeDict["X1"] 
    elif WSSizeDict["X1ym"] < 0:
        raise ValueError("寸法X1ymの設定が不適切です")
    elif WSSizeDict["X1ym"] > WSSizeDict["X1"] or WSSizeDict["X1ym"] == "":
        WSSizeDict["X1ym"] = WSSizeDict["X1"]       
        
    # X3yp
    if "X3yp" not in WSSizeDict:
        WSSizeDict["X3yp"] = WSSizeDict["X3"] 
    elif WSSizeDict["X3yp"] < 0:
        raise ValueError("寸法X3ypの設定が不適切です")            
    elif WSSizeDict["X3yp"] > WSSizeDict["X3"] or WSSizeDict["X3yp"] == "":
        WSSizeDict["X3yp"] = WSSizeDict["X3"]   
        
    # X3ym
    if "X3ym" not in WSSizeDict:
        WSSizeDict["X3ym"] = WSSizeDict["X3"] 
    elif WSSizeDict["X3ym"] < 0:
        raise ValueError("寸法X3ymの設定が不適切です")               
    elif WSSizeDict["X3ym"] > WSSizeDict["X3"] or WSSizeDict["X3ym"] == "":
        WSSizeDict["X3ym"] = WSSizeDict["X3"]   
        
    # Y1xp
    if "Y1xp" not in WSSizeDict:
        WSSizeDict["Y1xp"] = WSSizeDict["Y1"]
    elif WSSizeDict["Y1xp"] < 0:
        raise ValueError("寸法Y1xpの設定が不適切です")     
    elif WSSizeDict["Y1xp"] > WSSizeDict["Y1"] or WSSizeDict["Y1xp"] == "":
        WSSizeDict["Y1xp"] = WSSizeDict["Y1"]  
        
    # Y1xm
    if "Y1xm" not in WSSizeDict:
        WSSizeDict["Y1xm"] = WSSizeDict["Y1"]  
    elif WSSizeDict["Y1xm"] < 0:
        raise ValueError("寸法Y1xmの設定が不適切です")                
    elif WSSizeDict["Y1xm"] > WSSizeDict["Y1"] or WSSizeDict["Y1xm"] == "":
        WSSizeDict["Y1xm"] = WSSizeDict["Y1"]  
        
    # Y3xp
    if "Y3xp" not in WSSizeDict:
        WSSizeDict["Y3xp"] = WSSizeDict["Y3"]
    elif WSSizeDict["Y3xp"] < 0:
        raise ValueError("寸法Y3xpの設定が不適切です")
    elif WSSizeDict["Y3xp"] > WSSizeDict["Y3"] or WSSizeDict["Y3xp"] == "":
        WSSizeDict["Y3xp"] = WSSizeDict["Y3"]
        
    # Y3xm
    if "Y3xm" not in WSSizeDict:
        WSSizeDict["Y3xm"] = WSSizeDict["Y3"]
    elif WSSizeDict["Y3xm"] < 0:
        raise ValueError("寸法Y3xmの設定が不適切です")            
    elif WSSizeDict["Y3xm"] > WSSizeDict["Y3"] or WSSizeDict["Y3xm"] == "":
        WSSizeDict["Y3xm"] = WSSizeDict["Y3"]
        
    WSSize = [WSSizeDict["X1"],  WSSizeDict["X2"],  WSSizeDict["X3"],
              WSSizeDict["X1yp"],WSSizeDict["X1ym"],WSSizeDict["X3yp"],WSSizeDict["X3ym"],
              WSSizeDict["Y1"],  WSSizeDict["Y2"],  WSSizeDict["Y3"],
              WSSizeDict["Y1xp"],WSSizeDict["Y1xm"],WSSizeDict["Y3xp"],WSSizeDict["Y3xm"],
              WSSizeDict["Zxp"], WSSizeDict["Zxm"], WSSizeDict["Zyp"], WSSizeDict["Zym"] ]
            
    return WSSize